In [2]:
#load packages and comet key
from comet_ml import Experiment
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn import svm

cometkey = pd.read_csv('/home/polabs1/Downloads/cometml-API-key.csv')
experiment = Experiment(api_key=cometkey.loc[0, 'cometapikey'], project_name="Crypto_Dashboard")

np.random.seed(420)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/polabs/crypto-dashboard/a26c628f08894386b5cbbeed5431b50e



In [3]:
#load data
dfPr = pd.read_csv("/home/polabs1/Crypto/12.09.2018/train sets/BTCUSDT-a2.csv")
dfNr = pd.read_csv("/home/polabs1/Crypto/12.09.2018/train sets/BTCUSDT-a5.csv")

dfPt = pd.read_csv("/home/polabs1/Crypto/12.09.2018/test sets/BTCUSDT-a1.csv")
dfNt = pd.read_csv("/home/polabs1/Crypto/12.09.2018/test sets/BTCUSDT-a5.csv")

print('Positive 1% train shape')
print(dfPr.shape)
print('Positive 1% test shape')
print(dfPt.shape)

print('Negative 1% train shape')
print(dfNr.shape)
print('Negative 1% test shape')
print(dfNt.shape)

#Positive !% 24hr only for now
XY_train = np.split(dfPr, [1], axis=1)
XY_test = np.split(dfPt, [1], axis=1)


Positive 1% train shape
(4783, 50)
Positive 1% test shape
(568, 50)
Negative 1% train shape
(4783, 50)
Negative 1% test shape
(568, 50)


In [21]:
#1. Logistic regression
params={"scale":"Standard scale of test",
       "penalty":"L1",
       "C":1,
       "tol":0.1,
       "solver":"liblinear"}

scaler = preprocessing.StandardScaler().fit(XY_train[1])
XY_train_scale = scaler.transform(XY_train[1])        #print(pd.DataFrame(XY_train_scale).head())
XY_test_scale = scaler.transform(XY_test[1])        #print(pd.DataFrame(XY_test_scale).head())
        
Model_logistic = LogisticRegression(C=1, penalty='l1', tol=0.1, solver='liblinear')
Model_logistic.fit(XY_train_scale, XY_train[0].values.ravel()) 
Model_logistic_pred = Model_logistic.predict(XY_test_scale) 
print("Mean squared error: %.2f" % mean_squared_error(XY_test[0], Model_logistic_pred))
print('Variance score: %.2f' % r2_score(XY_test[0], Model_logistic_pred)) # Explained variance score: 1 is perfect prediction
print(Model_logistic.score(XY_test_scale, XY_test[0]))
      
CV_logistic = cross_val_score(Model_logistic, XY_test_scale, XY_test[0].values.ravel(), cv=5, verbose=2) #scoring="" njobs=
CV_logistic
print("Accuracy: %0.2f (+/- %0.2f)" % (CV_logistic.mean(), CV_logistic.std() * 2))
acc = CV_logistic.mean()
    
metrics={"Accuracy":acc, "STDaccuracy":CV_logistic.std()*2}

experiment.log_parameters(params)
experiment.log_metrics(metrics)


Mean squared error: 0.66
Variance score: -1.77
0.3415492957746479
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
Accuracy: 0.82 (+/- 0.05)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [4]:
 #2. SVM
params={"scale":"Standard scale of test",
       "gamma":0.01,
       "C":0.1,
       "kernel":"rbf"}

scaler = preprocessing.StandardScaler().fit(XY_train[1])
XY_train_scale = scaler.transform(XY_train[1])        #print(pd.DataFrame(XY_train_scale).head())
XY_test_scale = scaler.transform(XY_test[1])        #print(pd.DataFrame(XY_test_scale).head())
    
Model_svm = svm.SVC(gamma=0.01, C=0.1, kernel='rbf')#<-parameteres: C low is less penality of error term (generalizes better), higher C will overfit/stricter, gamma=degree of influence of single training example, low is neighborhood, high is far?
Model_svm.fit(XY_train_scale, XY_train[0].values.ravel())
Model_svm_pred = Model_svm.predict(XY_test_scale) # Make predictions using the testing set
print("Mean squared error: %.2f" % mean_squared_error(XY_test[0], Model_svm_pred))
print('Variance score: %.2f' % r2_score(XY_test[0], Model_svm_pred)) # Explained variance score: 1 is perfect prediction
print(Model_svm.score(XY_test_scale, XY_test[0]))
        
CV_svm = cross_val_score(Model_svm, XY_test_scale, XY_test[0].values.ravel(), cv=5, verbose=2) #scoring="" njobs=
CV_svm
print("Accuracy: %0.2f (+/- %0.2f)" % (CV_svm.mean(), CV_svm.std() * 2))
acc = CV_svm.mean()

metrics={"Accuracy":acc, "STDaccuracy":CV_svm.std()*2}

experiment.log_parameters(params)
experiment.log_metrics(metrics)

Mean squared error: 0.65
Variance score: -1.76
0.34507042253521125
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
Accuracy: 0.64 (+/- 0.04)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
